In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.cross_validation import train_test_split
import re
import datetime as dt
from sklearn import preprocessing

In [2]:
train = pd.read_csv('train_dataset.csv')
test = pd.read_csv('test_dataset.csv')

In [3]:
y = train.QuoteConversion_Flag.values
train = train.drop(['QuoteNumber', 'QuoteConversion_Flag'], axis=1)
test = test.drop('QuoteNumber', axis=1)

In [4]:
clf = xgb.XGBClassifier(n_estimators=25,
                        nthread=-1,
                        max_depth=10,
                        learning_rate=0.025,
                        silent=True,
                        subsample=0.8,
                        colsample_bytree=0.8)

xgb_model = clf.fit(train, y, eval_metric="auc")
preds = clf.predict_proba(test)[:, 1]

In [5]:
sample = pd.read_csv('sample_submission.csv')
sample.loc[:, 'QuoteConversion_Flag'] = preds
sample.to_csv('xgb_benchmark.csv', index=False)